<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Red_Neuronas_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción a Redes Neuronales con Keras

Este notebook está diseñado para demostrar cómo construir y entrenar un modelo simple de red neuronal utilizando la biblioteca Keras en TensorFlow. Usaremos un conjunto de datos muy básico de frases en español para clasificar su sentimiento como positivo o negativo.

##1) Cargamos y preparamos los datos

El conjunto de datos consiste en frases etiquetadas manualmente para simplificar el uso y la comprensión. Las frases son las siguientes:

- "Me gusta mucho este curso." -> Positivo   
- "Estoy aburrido de la rutina diaria." -> Negativo   
- "El clima hoy es maravilloso." -> Positivo   
- "No estoy satisfecho con el servicio." -> Negativo  

In [1]:
# Example sentences and their labels
sentences = ['Me gusta mucho este curso',
             'Estoy aburrido de la rutina diaria',
             'El clima hoy es maravilloso',
             'No estoy satisfecho con el servicio']
labels = [1, 0, 1, 0]  # 1: Positivo, 0: Negativo

Transformamos el texto en números mediante la técnica ** one-hot encoding** que convierte un texto en una lista de índices enteros, donde cada entero representa una palabra en el texto, codificada mediante una técnica simple de hashing

In [2]:
from tensorflow.keras.preprocessing.text import one_hot

text = "hello world"
vocab_size = 10000

# Codificar el texto
result = one_hot(text, vocab_size)
print(result)


[4381, 1425]


Tenemos también que estandarizar la longitud de las secuencias de texto (en este caso, los índices enteros que representan palabras) para que todas tengan el mismo tamaño. Este paso es necesario para entrenar la mayoría de los modelos de aprendizaje profundo, especialmente aquellos que involucran capas que esperan un tamaño de entrada fijo:

In [3]:
encoded_sentences = [[1, 3, 5], [7, 9], [10, 20, 30, 40, 50]]

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sentences = pad_sequences(encoded_sentences, maxlen=10, padding='post')
print(padded_sentences)

[[ 1  3  5  0  0  0  0  0  0  0]
 [ 7  9  0  0  0  0  0  0  0  0]
 [10 20 30 40 50  0  0  0  0  0]]


A continuación preparamos los datos de entrenamiento para tener representaciones numéricas del mismo tamaño:

In [5]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare(corpus, vocab_size=50, max_length = 10):
  encoded_sentences = []
  for sentence in corpus:
    # One-hot encode sentences
    encoded_sentences.append(one_hot(sentence, vocab_size))
  # Pad sequences to ensure uniform input size
  prepared_sentences = pad_sequences(encoded_sentences, maxlen=max_length, padding='post')
  return prepared_sentences

vocab_size = 50
max_length = 10
prepared_sentences = prepare(sentences, vocab_size, max_length)
print(prepared_sentences)

[[18 19 19 33 22  0  0  0  0  0]
 [26 11 29 42 46 35  0  0  0  0]
 [22 34 40 14 11  0  0  0  0  0]
 [14 26  2 19 22 28  0  0  0  0]]


##2) Creamos el modelo basado en una red de neuronas

A continuación, configuraremos un modelo `Sequential`. Este modelo es una pila lineal de capas. Podemos añadir capas con el método `add` y experimentar con diferentes arquitecturas cambiando el número y tipo de capas o ajustando parámetros como el número de neuronas por capa o las funciones de activación.

In [6]:
# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
vector_size = 8
model.add(Embedding(vocab_size, vector_size))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

print("Red diseñada correctamente")

Red diseñada correctamente


Compilamos el modelo seleccionando:
- **optimizador (optimizer)**: encargado de cambiar los atributos de la red neuronal como los pesos y la tasa de aprendizaje para reducir las pérdidas (e.g. 'adam', o 'rmsprop')
- **función de pérdida (loss)**: mide como de bien el modelo está haciendo sus predicciones comparadas con los valores reales. El objetivo del entrenamiento es minimizar esta función.
- **evaluación (metrics)**:  métricas utilizadas para evaluar el rendimiento del modelo. No se utilizan para entrenar el modelo pero son importantes para analizar cómo está funcionando el modelo.

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##3) Entrenamos el modelo

In [8]:
import numpy as np

# Convertir a NumPy arrays para asegurar compatibilidad y rendimiento
prepared_sentences = np.array(prepared_sentences)
labels = np.array(labels)

# Entrenamiento del modelo
model.fit(prepared_sentences, labels, epochs=10)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 0.7056
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.2500 - loss: 0.7026
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.2500 - loss: 0.6996
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.2500 - loss: 0.6967
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 0.6938
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 0.6908
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7500 - loss: 0.6879
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7500 - loss: 0.6850
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 0.6821
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 1.0000 - loss: 0.6793


##4) Realizamos algunas predicciones

In [9]:
# Conjunto más amplio de frases de prueba
test_sentences = [
    "no fui al último concierto pq nadie me quería acompañar",
    "Envidio de buena manera a los que tienen la oportunidad de ir mañana al estadio",
    "Se nos está volviendo costumbre del domingo por la noche, ver el episodio anterior de SNL y eso me hace recibir el lunes en un mejor mood",
    "hoy te he visto (y tu a mi) y no me has saludado, mala gente  (no te dije nada porque ibas acompañada)"
]

# Preparar los datos
prepared_test = prepare(test_sentences, vocab_size=vocab_size, max_length=max_length)

# Realizar predicciones
predictions = model.predict(prepared_test)

# Interpretar las predicciones con más detalle
print("Predicciones detalladas:")
for i, sentence in enumerate(test_sentences):
    pred = predictions[i][0]
    sentiment = "Positivo" if pred > 0.5 else "Negativo"
    print(f"\nTexto: {sentence}")
    print(f"Predicción numérica: {pred:.4f}")
    print(f"Sentimiento predicho: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicciones detalladas:

Texto: no fui al último concierto pq nadie me quería acompañar
Predicción numérica: 0.5214
Sentimiento predicho: Positivo

Texto: Envidio de buena manera a los que tienen la oportunidad de ir mañana al estadio
Predicción numérica: 0.5041
Sentimiento predicho: Positivo

Texto: Se nos está volviendo costumbre del domingo por la noche, ver el episodio anterior de SNL y eso me hace recibir el lunes en un mejor mood
Predicción numérica: 0.5149
Sentimiento predicho: Positivo

Texto: hoy te he visto (y tu a mi) y no me has saludado, mala gente  (no te dije nada porque ibas acompañada)
Predicción numérica: 0.4795
Sentimiento predicho: Negativo


# 5) Mejoras

## 5.1) Aumentar el conjunto de entrenamiento

Probemos con el dataset "[cardiffnlp/tweet_sentiment_multilingual](https://huggingface.co/datasets/cardiffnlp/tweet_sentiment_multilingual)" que contiene tweets en varios idiomas, incluido español

In [10]:
!pip install datasets

In [11]:
from datasets import load_dataset
import numpy as np

# Cargar el dataset especificando la configuración para español
dataset = load_dataset('cardiffnlp/tweet_sentiment_multilingual', 'spanish')

# Convertir a formato binario
new_sentences = []
new_labels = []

# Procesamos el conjunto de entrenamiento
for item in dataset['train']:
    # Convertir las etiquetas (0=negativo, 2=positivo)
    if item['label'] != 1:  # Excluimos neutral (1)
        text = item['text']
        # Convertimos 2 (positivo) a 1, 0 (negativo) se queda como 0
        sentiment = 1 if item['label'] == 2 else 0
        new_sentences.append(text)
        new_labels.append(sentiment)

# Convertir labels a lista primero
labels_list = labels.tolist() if isinstance(labels, np.ndarray) else list(labels)

# Combinar las listas
all_sentences = list(sentences) + new_sentences
all_labels = labels_list + new_labels

# Ahora convertir todo a numpy arrays
all_sentences = np.array(all_sentences)
all_labels = np.array(all_labels)

# Mezclar aleatoriamente los datos
indices = np.arange(len(all_labels))
np.random.shuffle(indices)
all_sentences = all_sentences[indices]
all_labels = all_labels[indices]

print(f"Tamaño total del dataset: {len(all_sentences)}")
print(f"Distribución de etiquetas: Positivos={np.sum(all_labels == 1)}, Negativos={np.sum(all_labels == 0)}")

# Ver algunas frases de ejemplo
print("\nEjemplos del dataset mezclado:")
for i in range(5):
    print(f"Texto: {all_sentences[i]}")
    print(f"Sentimiento: {'Positivo' if all_labels[i] == 1 else 'Negativo'}\n")

# Preparar los datos para el modelo
prepared_sentences = prepare(all_sentences, vocab_size, max_length)
labels = all_labels

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tamaño total del dataset: 1230
Distribución de etiquetas: Positivos=615, Negativos=615

Ejemplos del dataset mezclado:
Texto: @user No sé si está traducido, yo lo busqué hace tiempo y sólo lo encontré subtitulado
Sentimiento: Negativo

Texto: @user @user bienvenido al lugar en el que una bufanda nunca sobra
Sentimiento: Positivo

Texto: De nuevo lo veo imposible con tanta mención @user #FollowSpree. Por favor, mi apoyo lo tienes siempre
Sentimiento: Positivo

Texto: @user te tienes que hacer amiga de peluqueros que no estén locos y fuera, tienes don de gentes coñe
Sentimiento: Positivo

Texto: me siento orgullosa de ser tu fan @user desde hace muchos años ...ese speach lo dijo todo te sigo admirando
Sentimiento: Positivo



##5.2) Uso de Embeddings Preentrenados en vez de one-hot encoding

Existen varios embeddings preentrenados disponibles que se podrían utilizar. Algunos de los más populares incluyen:

- Word2Vec: Entrenado en Google News dataset, disponible en varios tamaños.
   
- GloVe (Global Vectors for Word Representation): Disponible en varios tamaños y entrenado en diferentes corpus como Wikipedia o Twitter.

- FastText: Ofrecido por Facebook, similar a Word2Vec pero también considera subpalabras.

###5.2.1) Descarga de embeddings en español

In [12]:
import requests
import gzip
import shutil

# Descargar la versión comprimida de FastText en español (300MB en lugar de varios GB)
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec"
print("Descargando embeddings...")
response = requests.get(url)

# Guardar el archivo
with open("wiki.es.vec", "wb") as f:
    f.write(response.content)

print("Embeddings descargados")

Descargando embeddings...
Embeddings descargados


### 5.2.2) Cargar los Embeddings
Primero, hay que descargar los embeddings y cargarlos en nuestro entorno.

In [13]:
from gensim.models import KeyedVectors
import numpy as np

# Cargamos solo las 10,000 palabras más comunes para ahorrar memoria
embeddings = KeyedVectors.load_word2vec_format('wiki.es.vec', limit=10000)

### 5.2.3) Preparar la Matriz de Embeddings de los datos de entrenamiento
Vamos a crear una matriz de embeddings para usar en la capa de Embedding de Keras. Esta matriz debe tener un vector para cada palabra del vocabulario:

In [14]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

# Primero creamos un tokenizador
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sentences)

# Ahora obtenemos el word_index
word_index = tokenizer.word_index

# Crear la matriz de embeddings
embedding_dim = 300  # FastText usa 300 dimensiones
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i >= vocab_size:
        continue
    try:
        embedding_vector = embeddings[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

print("Matriz de embeddings creada")

Matriz de embeddings creada


### 5.2.4) Diseñar el Modelo con la Capa de Embedding Preentrenada
Ahora ya podemos crear el modelo utilizando la matriz de embeddings en la capa de Embedding. Es importante establecer trainable=False para no modificar los embeddings durante el entrenamiento:

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

A continuación puedes volver al paso 3 para entrenar el modelo

## 5.3) Añadir más capas

Agregar más capas a un modelo Sequential es sencillo. Basta con usar el método .add() para incluir nuevas capas. A continuación se añade una capa densa adicional y una capa de dropout para regularización. Hay que tener cuidado con no tener sobreajuste:

In [16]:
from tensorflow.keras.layers import Dropout

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadimos una nueva capa densa con activación ReLU
model.add(Dense(16, activation='relu'))

# Añadimos una capa de dropout para regularización
model.add(Dropout(0.3))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

##5.4) Cambiando las funciones de activación

Cambiar la función de activación es tan simple como actualizar el argumento de activación en las capas que lo permitan. Por ejemplo, para cambiar la función de activación de la nueva capa densa a tanh, se puede hacer así:

In [17]:
# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Cambiando a tanh
model.add(Dense(16, activation='tanh'))

# Añadimos una capa de dropout para regularización
model.add(Dropout(0.3))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

#6) RNN

Para implementar un modelo que utilice redes neuronales recurrentes en Keras, basta con incluir capas como SimpleRNN, LSTM (Long Short-Term Memory), o GRU (Gated Recurrent Units), que son diseñadas para manejar dependencias de secuencias a lo largo del tiempo:

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# # SimpleRNN con 64 unidades
model.add(SimpleRNN(units=64))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

###7.1) LSTM

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# LSTM con 50 unidades
model.add(LSTM(units=50))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

###7.2)GRU

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# # GRU con 50 unidades
model.add(GRU(units=50))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)